# LUAD CPTAC analysis

In [1]:
import pandas as pd
import numpy as np
from msresist.pre_processing import preprocessing, preprocessCPTAC
from msresist.sequence_analysis import pYmotifs
from msresist.clustering import MassSpecClustering

In [2]:
# pd.set_option('display.max_colwidth', 1000)
# pd.set_option('display.max_rows', 1000000)
pd.set_option('display.max_columns', 1000)

## 1 - Import and Preprocess Data

In [3]:
X = preprocessing(CPTAC=True, log2T=True)

1096/52428 peptides were not found in the proteome.
51332


/home/marcc/resistance-MS/msresist/sequence_analysis.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Gene'] = names
/home/marcc/resistance-MS/msresist/sequence_analysis.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Sequence'] = motifs


In [5]:
d = X.select_dtypes(include=['float64']).T
i = X.select_dtypes(include=['object'])

In [6]:
distance_method = "Binomial"
ncl = 4
GMMweight = 2

MSC = MassSpecClustering(i, ncl, GMMweight=GMMweight, distance_method=distance_method).fit(d, "NA")

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').